## Goal

The goal of this notebook is to develop a binary classifier of reviews on IMDB using the keras framework.

In [1]:
from keras.datasets import imdb

# The argument num_words=10000 means you’ll only keep the top 10,000 most frequently occurring words 
# in the training data. Rare words will be discarded. 

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

Using TensorFlow backend.


In [2]:
len(train_data)

25000

In [3]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [4]:
print(train_labels[0])

1


In [5]:
max(max(seq) for seq in train_data)

9999

## Word Index

In [6]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for key, value in word_index.items()])

decoded_review = ' '.join([reverse_word_index.get(w - 3, '?') for w in train_data[0]])

decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [7]:
min(word_index.values()), max(word_index.values())

(1, 88584)

In [8]:
reverse_word_index[1], reverse_word_index[2], reverse_word_index[3]

('the', 'and', 'a')

# One Hot Encoding 

In [9]:
import numpy as np


def one_hot_encode(sequences: np.array, dim: int = 10000) -> np.array:
    ret = np.zeros((sequences.shape[0], dim))
    for i, s in enumerate(sequences):
        ret[i, s] = 1 
    return ret

In [10]:
x_train = one_hot_encode(train_data)
x_test = one_hot_encode(test_data)

In [11]:
# the labels are already vectorized

y_train = train_labels.astype('float32')
y_test = test_labels.astype('float32')

In [12]:
x_train.shape, y_train.shape

((25000, 10000), (25000,))

In [13]:
x_test.shape, y_test.shape

((25000, 10000), (25000,))

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [19]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 3s 178us/step - loss: 0.4982 - acc: 0.7726 - val_loss: 0.3619 - val_acc: 0.8619
Epoch 2/20
15000/15000 [==============================] - 2s 127us/step - loss: 0.2826 - acc: 0.8997 - val_loss: 0.2857 - val_acc: 0.8924
Epoch 3/20
15000/15000 [==============================] - 2s 148us/step - loss: 0.2014 - acc: 0.9309 - val_loss: 0.2859 - val_acc: 0.8851
Epoch 4/20
15000/15000 [==============================] - 2s 117us/step - loss: 0.1556 - acc: 0.9473 - val_loss: 0.2810 - val_acc: 0.8865
Epoch 5/20
15000/15000 [==============================] - 2s 121us/step - loss: 0.1323 - acc: 0.9545 - val_loss: 0.2931 - val_acc: 0.8863
Epoch 6/20
15000/15000 [==============================] - 2s 121us/step - loss: 0.1024 - acc: 0.9684 - val_loss: 0.3349 - val_acc: 0.8792
Epoch 7/20
15000/15000 [==============================] - 2s 154us/step - loss: 0.0818 - acc: 0.9745 - val_loss: 0.3398 - 

In [20]:
results = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 6s 246us/step


In [21]:
results

[0.8824923115968705, 0.8497999906539917]

In [26]:
model.get_weights()[0].shape

(10000, 32)

In [27]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)
results

Epoch 1/4
25000/25000 [==============================] - 2s 93us/step - loss: 0.4388 - accuracy: 0.8255
Epoch 2/4
25000/25000 [==============================] - 2s 98us/step - loss: 0.2511 - accuracy: 0.9114
Epoch 3/4
25000/25000 [==============================] - 2s 92us/step - loss: 0.1938 - accuracy: 0.9298
Epoch 4/4
25000/25000 [==============================] - 5s 190us/step


[0.3004542882823944, 0.8835999965667725]